# ライブラリの読み込み

In [3]:
!pip install econml
!pip install -U DoubleML
!pip install linearmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.9/532.9 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.3/237.3 kB 20.9 MB/s eta 0:00:00
  Attempting uninstall: slicer
    Found existing installation: slicer 0.0.8
    Uninstalling slicer-0.0.8:
      Successfully uninstalled slicer-0.0.8
  Attempting uninstall: shap
    Found existing installation: shap 0.46.0
    Uninstalling shap-0.46.0:
      Successfully uninstalled shap-0.46.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.5/94.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.9 MB/s eta 0:00:00


In [4]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from doubleml.datasets import fetch_bonus
from doubleml import DoubleMLData
from sklearn.base import clone
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoCV
from doubleml import DoubleMLPLR
import statsmodels.api as sm
from linearmodels import PanelOLS

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# DATAの読み込み

In [101]:
data1 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/data/data.xlsx')
data2 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/data/add_data.xlsx')

In [102]:
merged_data = pd.merge(data1, data2, left_index=True, right_index=True, how='inner')

In [103]:
merged_data.rename(columns={'docID_x': 'docID', 'name_x': 'name', 'year_x': 'year', 'secCode_y': 'secCode'}, inplace=True)

In [104]:
data = merged_data[[
    'docID',
    'year',
    'secCode',
    '売上高',
    'ROE',
    '純資産額',
    '総資産額',
    '経常利益',
    '自己資本比率',
    'male_num',
    'female_num',
    '従業員数',
    'RandD',
    'filerName',
    'is_consolidated',
    'market_type',
    '33industry_code',
    '17industry_code',
    'scale_code']]

In [105]:
def convert_vals(data, column_name, value_type):
    data[column_name] = data[column_name].replace('－', None)
    data[column_name] = data[column_name].replace('-', None)
    data.loc[(~data[column_name].isna()), column_name] = data.loc[(~data[column_name].isna()), column_name].astype(value_type)
    return data

In [ ]:
column_names = ['year', '売上高', '総資産額', '純資産額', '総資産額', '経常利益', 'male_num', 'female_num', 'RandD', '従業員数']
for column_name in column_names:
    data = convert_vals(data, column_name, 'int')
data = data.replace({np.nan: None})

column_names = ['ROE', '自己資本比率']
for column_name in column_names:
    data = convert_vals(data, column_name, 'float')

In [119]:
16740 / 9

1860.0

## 欠損値処理

In [107]:
data.loc[3672, 'year'] = 2015
data.loc[3673, 'year'] = 2016
data.loc[3674, 'year'] = 2017
data.loc[3675, 'year'] = 2018
data.loc[3676, 'year'] = 2019
data.loc[3677, 'year'] = 2020
data.loc[9333, 'year'] = 2015

In [108]:
# 9895
data.loc[9895, '売上高'] = 54752724000

In [109]:
# 43
data.loc[43, 'male_num'] = 10

In [110]:
data.loc[(data['ROE'].isna()), 'ROE'] = data['経常利益'] / data['純資産額']
data['RandD'] = data['RandD'].fillna(data.groupby('secCode')['RandD'].transform('mean'))
data['RandD'] = data['RandD'].fillna(0)

<ipython-input-110-03e1b1a27298>:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['RandD'] = data['RandD'].fillna(data.groupby('secCode')['RandD'].transform('mean'))


# 変数の追加

In [111]:
panel_data = data.set_index(['year', 'secCode'])
panel_data = panel_data.sort_values(['secCode', 'year'])
panel_data['prev_売上高'] = panel_data.groupby('secCode')['売上高'].shift(1)
panel_data['売上高成長率'] = ((panel_data['売上高'] - panel_data['prev_売上高']) / panel_data['prev_売上高'])
data = panel_data.reset_index()

In [112]:
data['female_dummy'] = np.where(
    (data['female_num'] == 0),
    0,  # 条件を満たす場合
    1   # 条件を満たさない場合
)

data['RandD_dummy'] = np.where(
    (data['RandD'] == 0 | data['RandD'].isna()),
    0,  # 条件を満たす場合
    1   # 条件を満たさない場合
)

data['ROA'] = data['経常利益'] / data['総資産額']
data['female_ratio'] = data['female_num'] / (data['female_num'] + data['male_num'])

In [113]:
data['RandD_log'] = (data['RandD'])
data['RandD_log'] = data['RandD_log'].apply(lambda x: np.log(x) if x > 0 else 0)

In [114]:
data['RandD_log'] = data['RandD_log'].round(3)
data['RandD_log']

,RandD_log
0,19.421
1,19.406
2,19.450
3,19.485
4,19.533
...,...
16735,0.000
16736,0.000
16737,0.000
16738,0.000


# 異常値の処理

In [120]:
data_save = data.copy()

In [122]:
data[data['ROE'] < -2.5][['ROE', 'year', 'secCode',]]

,ROE,year,secCode
2013,-2.781243,2021,26860
2014,-2.982025,2022,26860
2598,-8.58022,2021,30530
2985,-3.727,2021,31750
2987,-5.35,2023,31750
9688,-2.565,2019,67400
9689,-4.061,2020,67400
11411,-5.9595,2023,75240
13057,-5.090693,2022,81070
13058,-6.15995,2023,81070


In [125]:
data[data['ROE'] > 2.5][['ROA', 'year', 'secCode',]]

,ROA,year,secCode
9034,-0.077572,2022,64440


In [126]:
data[data['売上高成長率'] > 3.0][['売上高成長率', 'year', 'secCode',]]

,売上高成長率,year,secCode
5694,10.256952,2021,45870
6284,3.012366,2017,48450
13479,8.394308,2021,82520
14717,4.078547,2017,89180


In [127]:
data[data['自己資本比率'] < 0][['自己資本比率', 'year', 'secCode',]]

,自己資本比率,year,secCode
2015,-0.078,2023,26860
2634,-0.031,2021,30730
9033,-0.119,2021,64440
9757,-0.027,2016,67530
12282,-0.431,2021,79180
14586,-0.0525,2021,88480


In [128]:
data = data.drop(data[data['secCode'].isin([26860, 30530, 31750, 64440, 67400, 75240, 81070])].index)

In [129]:
data = data.drop(data[data['secCode'].isin([45870, 82520])].index)

In [130]:
data = data.drop(data[data['secCode'].isin([79180])].index)

# 記述統計

## 4章

In [ ]:
data.columns

Index(['year', 'secCode', 'docID', '売上高', 'ROE', '純資産額', '総資産額', '経常利益',
       '自己資本比率', 'male_num', 'female_num', '従業員数', 'RandD', 'filerName',
       'is_consolidated', 'market_type', '33industry_code', '17industry_code',
       'prev_売上高', '売上高成長率', 'female_dummy', 'RandD_dummy', 'ROA',
       'female_ratio', 'RandD_log'],
      dtype='object')

In [131]:
data['ROA'] = data['ROA'] * 100
data['ROE'] = data['ROE'] * 100
data['売上高成長率'] = data['売上高成長率'] * 100
data['female_ratio'] = data['female_ratio'] * 100
data['自己資本比率'] = data['自己資本比率'] * 100

In [132]:
data = data[data['year'] != 2015]

In [134]:
data.to_pickle('/content/drive/MyDrive/Colab Notebooks/OutputData/using_data.pickle')

In [135]:
column_names = ['year', 'secCode','売上高', '純資産額', '総資産額', '経常利益', 'male_num', 'female_num', 'RandD', '従業員数', 'female_dummy', 'RandD_dummy', '33industry_code', '17industry_code',]
for column_name in column_names:
    data[column_name] = data[column_name].astype('int')

column_names = ['ROE', 'ROA', 'female_ratio', '売上高成長率', '自己資本比率']
for column_name in column_names:
    data[column_name] = data[column_name].astype('float')

In [ ]:
summary = data[['female_ratio', 'ROE', 'ROA','売上高成長率', '従業員数', '自己資本比率', 'RandD_log', '総資産額', '純資産額', '経常利益', 'male_num', 'female_num', 'female_dummy', 'RandD_dummy']].describe().transpose()
summary_result = summary.loc[: , ['mean', '50%', 'std', 'min', 'max']]
summary_result
# summary_result.to_excel('/content/drive/MyDrive/Colab Notebooks/OutputData/summary.xlsx')

In [ ]:
female_num_table = pd.crosstab(data['female_num'], data['year'])
female_num_table.to_excel('/content/drive/MyDrive/Colab Notebooks/OutputData/female_num_table.xlsx')

In [ ]:
table = pd.DataFrame()
column_names = ['ROE', 'ROA', '売上高成長率', '従業員数', '自己資本比率', 'RandD', '総資産額', '純資産額', '経常利益']
for column_name in column_names:
    for i in range(0, 8):
        tmp = data.loc[(data['female_num'] == i), column_name].mean()
        table.loc[i, column_name] = round(tmp, 3)

table
# table.to_excel('/content/drive/MyDrive/Colab Notebooks/OutputData/table.xlsx')

## 5章

In [138]:
industry_label = pd.DataFrame.from_dict({
    1: '食品',
    2: 'エネルギー資源',
    3: '建設・資材',
    4: '素材・化学',
    5: '医薬品',
    6: '自動車・輸送機',
    7: '鉄鋼・非鉄',
    8: '機械',
    9: '電機・精密',
    10: '情報通信・サービスその他',
    11: '電力・ガス',
    12: '運輸・物流',
    13: '商社・卸売',
    14: '小売',
    15: '銀行',
    16: '金融（除く銀行）',
    17: '不動産',
}, orient='index', columns=['industry'])

In [146]:
for i in range(1, 18):
    tmp = len(data.loc[data['17industry_code'] == i]) / 8
    industry_label.loc[i, 'num'] = int(tmp)

In [ ]:
table = pd.DataFrame()
column_names = ['ROE', 'ROA', '売上高成長率', '従業員数', '自己資本比率', 'RandD', '総資産額', '純資産額', '経常利益', 'female_ratio', 'female_num']
for column_name in column_names:
    for i in range(1, 18):
        tmp = data.loc[(data['17industry_code'] == i), column_name].mean()
        table.loc[i, column_name] = round(tmp, 3)

table['name'] = industry_label['industry']
table['count'] = industry_label['num']
# table.to_excel('/content/drive/MyDrive/Colab Notebooks/OutputData/table_industry.xlsx')
table

In [22]:
scale_label = pd.DataFrame.from_dict({
    1: 'TOPIX Core30',
    2: 'TOPIX Large70',
    4: 'TOPIX Mid400',
    6: 'TOPIX Small 1',
    7: 'TOPIX Small 2',
}, orient='index', columns=['scale_name'])

In [25]:
for i in [1, 2, 4, 6, 7]:
    tmp = len(data.loc[data['scale_code'] == i])
    scale_label.loc[i, 'num'] = int(tmp / 8)

scale_label

,scale_name,num
1,TOPIX Core30,28.0
2,TOPIX Large70,67.0
4,TOPIX Mid400,369.0
6,TOPIX Small 1,444.0
7,TOPIX Small 2,942.0


In [27]:
table = pd.DataFrame()
column_names = ['ROE', 'ROA', '売上高成長率', '従業員数', '自己資本比率', 'RandD', '総資産額', '純資産額', '経常利益', 'female_ratio', 'female_num']
for column_name in column_names:
    for i in [1, 2, 4, 6, 7]:
        tmp = data.loc[(data['scale_code'] == i), column_name].mean()
        table.loc[i, column_name] = round(tmp, 3)

table['name'] = scale_label['scale_name']
table['count'] = scale_label['num']

table.to_excel('/content/drive/MyDrive/Colab Notebooks/OutputData/table_scale.xlsx')
table

,ROE,ROA,売上高成長率,従業員数,自己資本比率,RandD,総資産額,純資産額,経常利益,female_ratio,female_num,name,count
1,15.374,8.999,7.057,93000.129,47.588,1.673673e+11,3.270007e+13,3.607877e+12,4.563555e+11,11.722,1.902,TOPIX Core30,28.0
2,11.547,7.312,5.725,53060.151,47.494,7.251901e+10,5.053676e+12,1.276229e+12,1.540385e+11,11.654,1.789,TOPIX Large70,67.0
4,9.497,7.221,4.977,14680.286,50.013,1.034372e+10,1.212769e+12,3.617543e+11,4.079548e+10,8.837,1.189,TOPIX Mid400,369.0
6,8.595,6.777,5.586,3689.598,52.484,1.840786e+09,4.270287e+11,9.646768e+10,9.907261e+09,7.114,0.828,TOPIX Small 1,444.0
7,6.865,5.738,4.040,1344.531,53.255,4.363073e+08,1.165580e+11,2.968123e+10,2.568798e+09,5.809,0.614,TOPIX Small 2,942.0


# データ分析

## OLS

In [ ]:
data.columns

Index(['year', 'secCode', 'docID', 'name', '売上高', 'ROE', '純資産額', '総資産額',
       '経常利益', '自己資本比率', 'male_num', 'female_num', '従業員数', 'RandD', 'prev_売上高',
       '売上高成長率', 'female_dummy', 'RandD_dummy', 'ROA', 'female_ratio',
       'female_str', 'RandD_log'],
      dtype='object')

In [ ]:
treatment_val   = data[['female_ratio', '自己資本比率', '従業員数', 'RandD_log', 'RandD_dummy', '売上高成長率']]
outcome_val     = data['ROE']
treatment_val   = sm.add_constant(treatment_val)

model = sm.OLS(outcome_val, treatment_val)
results = model.fit()

In [ ]:
# 有意性マークを付ける関数
def add_significance(stars, p_value):
    if p_value < 0.01:
        return f"{stars}***"
    elif p_value < 0.05:
        return f"{stars}**"
    elif p_value < 0.10:
        return f"{stars}*"
    else:
        return stars

# 回帰分析結果をデータフレームに変換して有意性マークを追加
def format_results(results):
    summary_df = pd.DataFrame({
        "coef": results.params,
        "std_err": results.bse,
        "t": results.tvalues,
        "P>|t|": results.pvalues
    })

    # 有効数字2桁にフォーマットして有意性マークを追加
    summary_df["coef"] = summary_df.apply(
        lambda x: add_significance(f"{round(x['coef'], 2):.2f}", x["P>|t|"]), axis=1
    )
    summary_df["std_err"] = summary_df["std_err"].apply(lambda x: f"{round(x, 2):.2f}")
    summary_df["t"] = summary_df["t"].apply(lambda x: f"{round(x, 2):.2f}")
    summary_df["P>|t|"] = summary_df["P>|t|"].apply(lambda x: f"{round(x, 2):.2f}")
    return summary_df

# 例: 回帰分析の結果を整形して表示
results_df = format_results(results)

In [ ]:
treatment_val   = data[['female_ratio', '自己資本比率', '従業員数', 'RandD_log', 'RandD_dummy', '売上高成長率']]
outcome_val     = data['ROA']
treatment_val   = sm.add_constant(treatment_val)

model = sm.OLS(outcome_val, treatment_val)
results2 = model.fit()

In [ ]:
# 例: 回帰分析の結果を整形して表示
results_df2 = format_results(results2)

In [ ]:
ols_result_table = pd.DataFrame()

In [ ]:
ols_result_table['ROE coef'] = results_df['coef']
ols_result_table['ROE t'] = results_df['t']

In [ ]:
ols_result_table['ROA coef'] = results_df2['coef']
ols_result_table['ROA t'] = results_df2['t']

In [ ]:
ols_result_table.to_excel('/content/drive/MyDrive/Colab Notebooks/OutputData/ols_result_table.xlsx')

## 固定効果モデル

In [ ]:
fixed_effects_table = pd.DataFrame()

In [ ]:
panel_data = data.set_index(['year', 'secCode'])
panel_data = panel_data.sort_values(['secCode', 'year'])

In [ ]:
# 説明変数と被説明変数
y = panel_data['ROE']
X = panel_data[['female_ratio', '自己資本比率', '従業員数', 'RandD_log', 'RandD_dummy', '売上高成長率']]

# 固定効果モデルのフィッティング
model = PanelOLS(y, X, entity_effects=True, drop_absorbed=True)
results3 = model.fit()

In [ ]:
# 有意性マークを付ける関数
def add_significance(stars, p_value):
    if p_value < 0.01:
        return f"{stars}***"
    elif p_value < 0.05:
        return f"{stars}**"
    elif p_value < 0.10:
        return f"{stars}*"
    else:
        return stars

# PanelOLS 結果を整形する関数
def format_panel_ols_results(results):
    summary_df = pd.DataFrame({
        "coef": results.params,
        "std_err": results.std_errors,
        "t": results.tstats,
        "P>|t|": results.pvalues
    })

    # 信頼区間を取得
    conf_int = results.conf_int()
    summary_df["95% Conf. Int. Lower"] = conf_int.iloc[:, 0]
    summary_df["95% Conf. Int. Upper"] = conf_int.iloc[:, 1]

    # 小数点以下2桁に丸めて有意性マークを追加
    summary_df["coef"] = summary_df.apply(
        lambda x: add_significance(f"{round(x['coef'], 2):.2f}", x["P>|t|"]), axis=1
    )
    summary_df["std_err"] = summary_df["std_err"].apply(lambda x: f"{round(x, 2):.2f}")
    summary_df["t"] = summary_df["t"].apply(lambda x: f"{round(x, 2):.2f}")
    summary_df["P>|t|"] = summary_df["P>|t|"].apply(lambda x: f"{round(x, 2):.2f}")
    summary_df["95% Conf. Int. Lower"] = summary_df["95% Conf. Int. Lower"].apply(lambda x: f"{round(x, 2):.2f}")
    summary_df["95% Conf. Int. Upper"] = summary_df["95% Conf. Int. Upper"].apply(lambda x: f"{round(x, 2):.2f}")

    return summary_df

# 固定効果モデルの結果を整形
results_df_ROE_panel = format_panel_ols_results(results3)
fixed_effects_table['ROE coef'] = results_df_ROE_panel['coef']
fixed_effects_table['ROE t'] = results_df_ROE_panel['t']

In [ ]:
# 説明変数と被説明変数
y = panel_data['ROA']
X = panel_data[['female_ratio', '自己資本比率', '従業員数', 'RandD_log', 'RandD_dummy', '売上高成長率']]

# 固定効果モデルのフィッティング
model = PanelOLS(y, X, entity_effects=True, drop_absorbed=True)
results4 = model.fit()
results_df_ROA_panel = format_panel_ols_results(results4)
fixed_effects_table['ROA coef'] = results_df_ROA_panel['coef']
fixed_effects_table['ROA t'] = results_df_ROA_panel['t']

In [ ]:
fixed_effects_table.to_excel('/content/drive/MyDrive/Colab Notebooks/OutputData/fixed_effects_table.xlsx')

## CRE with DML

In [35]:
dml_data_bonus = DoubleMLData(data,
                                y_col='ROE',
                                d_cols='female_dummy',
                                x_cols=['自己資本比率', '従業員数', 'RandD_log', 'RandD_dummy', '売上高成長率'])

learner = RandomForestRegressor(n_estimators = 500, max_features = 'sqrt', max_depth= 5)
ml_l_bonus = clone(learner)
ml_m_bonus = clone(learner)

obj_dml_plr_bonus = DoubleMLPLR(dml_data_bonus, ml_l_bonus, ml_m_bonus)
obj_dml_plr_bonus.fit();
print(obj_dml_plr_bonus)

================== DoubleMLPLR Object ==================

------------------ Data summary      ------------------
Outcome variable: ROE
Treatment variable(s): ['female_dummy']
Covariates: ['自己資本比率', '従業員数', 'RandD_log', 'RandD_dummy', '売上高成長率']
Instrument variable(s): None
No. Observations: 14800

------------------ Score & algorithm ------------------
Score function: partialling out

------------------ Machine learner   ------------------
Learner ml_l: RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=500)
Learner ml_m: RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=500)
Out-of-sample Performance:
Regression:
Learner ml_l RMSE: [[12.1089424]]
Learner ml_m RMSE: [[0.4720662]]

------------------ Resampling        ------------------
No. folds: 5
No. repeated sample splits: 1

------------------ Fit summary       ------------------
                 coef   std err         t     P>|t|     2.5 %    97.5 %
female_dummy  0.30404  0.219941  1.382374  0.

In [36]:
dml_data_bonus2 = DoubleMLData(data,
                                y_col='ROA',
                                d_cols='female_dummy',
                                x_cols=['自己資本比率', '従業員数', 'RandD_log', 'RandD_dummy', '売上高成長率'])

learner = RandomForestRegressor(n_estimators = 500, max_features = 'sqrt', max_depth= 5)
ml_l_bonus = clone(learner)
ml_m_bonus = clone(learner)

obj_dml_plr_bonus2 = DoubleMLPLR(dml_data_bonus2, ml_l_bonus, ml_m_bonus)
obj_dml_plr_bonus2.fit();
print(obj_dml_plr_bonus2)

================== DoubleMLPLR Object ==================

------------------ Data summary      ------------------
Outcome variable: ROA
Treatment variable(s): ['female_dummy']
Covariates: ['自己資本比率', '従業員数', 'RandD_log', 'RandD_dummy', '売上高成長率']
Instrument variable(s): None
No. Observations: 14800

------------------ Score & algorithm ------------------
Score function: partialling out

------------------ Machine learner   ------------------
Learner ml_l: RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=500)
Learner ml_m: RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=500)
Out-of-sample Performance:
Regression:
Learner ml_l RMSE: [[5.76058378]]
Learner ml_m RMSE: [[0.47210023]]

------------------ Resampling        ------------------
No. folds: 5
No. repeated sample splits: 1

------------------ Fit summary       ------------------
                  coef   std err         t     P>|t|     2.5 %    97.5 %
female_dummy  0.286106  0.098529  2.903782 

## 追加分析

In [28]:
data = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/OutputData/using_data.pickle')

In [32]:
data.columns

Index(['year', 'secCode', 'docID', '売上高', 'ROE', '純資産額', '総資産額', '経常利益',
       '自己資本比率', 'male_num', 'female_num', '従業員数', 'RandD', 'filerName',
       'is_consolidated', 'market_type', '33industry_code', '17industry_code',
       'scale_code', 'prev_売上高', '売上高成長率', 'female_dummy', 'RandD_dummy',
       'ROA', 'female_ratio', 'RandD_log', 'industry_1', 'industry_2',
       'industry_3', 'industry_4', 'industry_5', 'industry_6', 'industry_7',
       'industry_8', 'industry_9', 'industry_10', 'industry_11', 'industry_12',
       'industry_13', 'industry_14', 'industry_15', 'industry_16',
       'industry_17', 'scale_1', 'scale_2', 'scale_4', 'scale_6', 'scale_7'],
      dtype='object')

In [30]:
dummy_data = pd.get_dummies(data['17industry_code'], prefix='industry')
data = pd.concat([data, dummy_data], axis=1)

In [31]:
dummy_data = pd.get_dummies(data['scale_code'], prefix='scale')
data = pd.concat([data, dummy_data], axis=1)

In [33]:
dml_data_bonus = DoubleMLData(data,
                                y_col='ROE',
                                d_cols='female_dummy',
                                x_cols=['自己資本比率', '従業員数', 'RandD_log', 'RandD_dummy', '売上高成長率',
                                        'industry_1', 'industry_2', 'industry_3', 'industry_4', 'industry_5', 'industry_6', 'industry_7', 'industry_8', 'industry_9',
                                        'industry_10', 'industry_11', 'industry_12', 'industry_13', 'industry_14', 'industry_15', 'industry_16', 'industry_17',
                                        'scale_1', 'scale_2', 'scale_4', 'scale_6', 'scale_7',
                                        ])

learner = RandomForestRegressor(n_estimators = 500, max_features = 'sqrt', max_depth= 5)
ml_l_bonus = clone(learner)
ml_m_bonus = clone(learner)

obj_dml_plr_bonus = DoubleMLPLR(dml_data_bonus, ml_l_bonus, ml_m_bonus)
obj_dml_plr_bonus.fit();
print(obj_dml_plr_bonus)

================== DoubleMLPLR Object ==================

------------------ Data summary      ------------------
Outcome variable: ROE
Treatment variable(s): ['female_dummy']
Covariates: ['自己資本比率', '従業員数', 'RandD_log', 'RandD_dummy', '売上高成長率', 'industry_1', 'industry_2', 'industry_3', 'industry_4', 'industry_5', 'industry_6', 'industry_7', 'industry_8', 'industry_9', 'industry_10', 'industry_11', 'industry_12', 'industry_13', 'industry_14', 'industry_15', 'industry_16', 'industry_17', 'scale_1', 'scale_2', 'scale_4', 'scale_6', 'scale_7']
Instrument variable(s): None
No. Observations: 14800

------------------ Score & algorithm ------------------
Score function: partialling out

------------------ Machine learner   ------------------
Learner ml_l: RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=500)
Learner ml_m: RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=500)
Out-of-sample Performance:
Regression:
Learner ml_l RMSE: [[12.01013063]]
Learn

In [34]:
dml_data_bonus = DoubleMLData(data,
                                y_col='ROA',
                                d_cols='female_dummy',
                                x_cols=['自己資本比率', '従業員数', 'RandD_log', 'RandD_dummy', '売上高成長率',
                                        'industry_1', 'industry_2', 'industry_3', 'industry_4', 'industry_5', 'industry_6', 'industry_7', 'industry_8', 'industry_9',
                                        'industry_10', 'industry_11', 'industry_12', 'industry_13', 'industry_14', 'industry_15', 'industry_16', 'industry_17',
                                        'scale_1', 'scale_2', 'scale_4', 'scale_6', 'scale_7',
                                        ])

learner = RandomForestRegressor(n_estimators = 500, max_features = 'sqrt', max_depth= 5)
ml_l_bonus = clone(learner)
ml_m_bonus = clone(learner)

obj_dml_plr_bonus = DoubleMLPLR(dml_data_bonus, ml_l_bonus, ml_m_bonus)
obj_dml_plr_bonus.fit();
print(obj_dml_plr_bonus)

================== DoubleMLPLR Object ==================

------------------ Data summary      ------------------
Outcome variable: ROA
Treatment variable(s): ['female_dummy']
Covariates: ['自己資本比率', '従業員数', 'RandD_log', 'RandD_dummy', '売上高成長率', 'industry_1', 'industry_2', 'industry_3', 'industry_4', 'industry_5', 'industry_6', 'industry_7', 'industry_8', 'industry_9', 'industry_10', 'industry_11', 'industry_12', 'industry_13', 'industry_14', 'industry_15', 'industry_16', 'industry_17', 'scale_1', 'scale_2', 'scale_4', 'scale_6', 'scale_7']
Instrument variable(s): None
No. Observations: 14800

------------------ Score & algorithm ------------------
Score function: partialling out

------------------ Machine learner   ------------------
Learner ml_l: RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=500)
Learner ml_m: RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=500)
Out-of-sample Performance:
Regression:
Learner ml_l RMSE: [[5.75158251]]
Learne